In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold
from sklearn.linear_model  import LogisticRegression
from sklearn.metrics import accuracy_score,log_loss
import numpy as np
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

In [2]:
image=pd.read_csv("Image_Segmention.csv")
image.head()

,Class,region.centroid.col,region.centroid.row,region.pixel.count,short.line.density.5,short.line.density.2,vedge.mean,vegde.sd,hedge.mean,hedge.sd,intensity.mean,rawred.mean,rawblue.mean,rawgreen.mean,exred.mean,exblue.mean,exgreen.mean,value.mean,saturation.mean,hue-mean
0,BRICKFACE,188,133,9,0.0,0.0,0.333333,0.266667,0.500000,0.077778,6.666666,8.333334,7.777778,3.888889,5.000000,3.333333,-8.333333,8.444445,0.538580,-0.924817
1,BRICKFACE,105,139,9,0.0,0.0,0.277778,0.107407,0.833333,0.522222,6.111111,7.555555,7.222222,3.555556,4.333334,3.333333,-7.666666,7.555555,0.532628,-0.965946
2,BRICKFACE,34,137,9,0.0,0.0,0.500000,0.166667,1.111111,0.474074,5.851852,7.777778,6.444445,3.333333,5.777778,1.777778,-7.555555,7.777778,0.573633,-0.744272
3,BRICKFACE,39,111,9,0.0,0.0,0.722222,0.374074,0.888889,0.429629,6.037037,7.000000,7.666666,3.444444,2.888889,4.888889,-7.777778,7.888889,0.562919,-1.175773
4,BRICKFACE,16,128,9,0.0,0.0,0.500000,0.077778,0.666667,0.311111,5.555555,6.888889,6.666666,3.111111,4.000000,3.333333,-7.333334,7.111111,0.561508,-0.985811


In [3]:
image["Class"].value_counts()

Class
SKY          30
FOLIAGE      30
CEMENT       30
PATH         30
WINDOW       30
GRASS        30
BRICKFACE    29
Name: count, dtype: int64

In [4]:
lbl=LabelEncoder()
image['Class']=lbl.fit_transform(image['Class'])
lbl.classes_

array(['BRICKFACE', 'CEMENT', 'FOLIAGE', 'GRASS', 'PATH', 'SKY', 'WINDOW'],
      dtype=object)

In [5]:
image.tail()

,Class,region.centroid.col,region.centroid.row,region.pixel.count,short.line.density.5,short.line.density.2,vedge.mean,vegde.sd,hedge.mean,hedge.sd,intensity.mean,rawred.mean,rawblue.mean,rawgreen.mean,exred.mean,exblue.mean,exgreen.mean,value.mean,saturation.mean,hue-mean
204,3,36,243,9,0.111111,0.0,1.888889,1.851851,2.000000,0.711110,13.333333,9.888889,12.111111,18.000000,-10.333333,-3.666667,14.000000,18.000000,0.452229,2.368311
205,3,186,218,9,0.000000,0.0,1.166667,0.744444,1.166667,0.655555,13.703704,10.666667,12.666667,17.777779,-9.111111,-3.111111,12.222222,17.777779,0.401347,2.382684
206,3,197,236,9,0.000000,0.0,2.444444,6.829628,3.333333,7.599998,16.074074,13.111111,16.666668,18.444445,-8.888889,1.777778,7.111111,18.555555,0.292729,2.789800
207,3,208,240,9,0.111111,0.0,1.055556,0.862963,2.444444,5.007407,14.148149,10.888889,13.000000,18.555555,-9.777778,-3.444444,13.222222,18.555555,0.421621,2.392487
208,3,223,185,9,0.000000,0.0,0.500000,0.349603,2.388889,2.080776,12.962963,11.555555,9.777778,17.555555,-4.222222,-9.555555,13.777778,17.555555,0.445418,1.838850


In [6]:
X=image.drop("Class",axis=1)
y=image['Class']

In [7]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=23,stratify=y)
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_pred_prob = lr.predict_proba(X_test)
print(log_loss(y_test,y_pred_prob))

0.27163401168459117


In [8]:
y_pred=lr.predict(X_test)
accuracy_score(y_test,y_pred)

0.9206349206349206

In [9]:
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=23)

In [19]:
penalty={'penalty':['l1', 'l2', 'elasticnet', None],
         'solver':['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
        'multi_class':['ovr','multinomial']}

In [31]:
gcv=GridSearchCV(lr,param_grid=penalty,cv=kfold,scoring='neg_log_loss')

In [40]:
gcv.fit(X,y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=LogisticRegression(),
             param_grid={'multi_class': ['ovr', 'multinomial'],
                         'penalty': ['l1', 'l2', 'elasticnet', None],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']},
             scoring='neg_log_loss')

In [41]:
print(gcv.best_params_)

{'multi_class': 'multinomial', 'penalty': 'l2', 'solver': 'newton-cg'}


In [42]:
print(gcv.best_score_)

-0.5465525609071172


In [43]:
pd.DataFrame(gcv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_multi_class,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001747,0.000189,0.000000,0.000000,ovr,l1,lbfgs,"{'multi_class': 'ovr', 'penalty': 'l1', 'solve...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,22
1,0.049889,0.010970,0.004099,0.000363,ovr,l1,liblinear,"{'multi_class': 'ovr', 'penalty': 'l1', 'solve...",-0.427192,-0.452920,-1.610347,-0.448237,-0.362938,-0.660327,0.476095,4
2,0.000758,0.000048,0.000000,0.000000,ovr,l1,newton-cg,"{'multi_class': 'ovr', 'penalty': 'l1', 'solve...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,22
3,0.000835,0.000123,0.000000,0.000000,ovr,l1,newton-cholesky,"{'multi_class': 'ovr', 'penalty': 'l1', 'solve...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,22
4,0.000706,0.000019,0.000000,0.000000,ovr,l1,sag,"{'multi_class': 'ovr', 'penalty': 'l1', 'solve...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,22
5,0.080519,0.024219,0.004349,0.000724,ovr,l1,saga,"{'multi_class': 'ovr', 'penalty': 'l1', 'solve...",-1.109199,-1.146707,-0.881940,-1.296280,-1.145575,-1.115940,0.133528,18
6,0.144446,0.005706,0.004288,0.000236,ovr,l2,lbfgs,"{'multi_class': 'ovr', 'penalty': 'l2', 'solve...",-0.409491,-0.414785,-1.682036,-0.423852,-0.435105,-0.673054,0.504566,5
7,0.014848,0.001047,0.003394,0.000139,ovr,l2,liblinear,"{'multi_class': 'ovr', 'penalty': 'l2', 'solve...",-0.467874,-0.455196,-1.638792,-0.431620,-0.399559,-0.678608,0.480655,6
8,0.307421,0.009141,0.004172,0.000128,ovr,l2,newton-cg,"{'multi_class': 'ovr', 'penalty': 'l2', 'solve...",-0.266945,-0.353149,-1.589314,-0.425868,-0.388789,-0.604813,0.495059,3
9,0.038193,0.002591,0.004150,0.000497,ovr,l2,newton-cholesky,"{'multi_class': 'ovr', 'penalty': 'l2', 'solve...",-0.266542,-0.353148,-1.589320,-0.425830,-0.388790,-0.604726,0.495120,2


In [47]:
model = gcv.best_estimator_
unlabel = pd.read_csv('tst_img.csv')
print(model.coef_)
print(model.intercept_)

[[-3.02403569e-02 -1.29097151e-01  1.05406506e-03 -1.64282484e-02
  -4.54631921e-05 -2.65706121e-01 -1.25123165e-01  3.68222555e-02
  -9.53361863e-02 -7.55518396e-02  2.68787652e-01 -3.37518384e-02
  -4.61691898e-01  1.03301938e+00  1.25400437e-01 -1.15841949e+00
  -5.48293635e-02 -2.48404226e-02  7.18655748e-02]
 [-5.85826462e-03 -5.97605861e-02 -1.98316736e-04  1.14007534e-01
   7.06925154e-02 -8.39163922e-02  1.49268512e-01  2.94176341e-01
  -7.06496922e-02  1.37638236e-01  2.77752016e-01  2.32265216e-01
  -9.71032078e-02  4.20339034e-01  2.83883328e-01 -7.04223218e-01
  -5.77254182e-01  2.11481520e-02 -1.78948045e-01]
 [-8.98890654e-03 -1.69606871e-01 -8.21690572e-04 -1.92327700e-02
   1.27682667e-03  2.86385357e-01  6.23072934e-02 -5.11576619e-02
   4.79211953e-02 -2.75448329e-01 -5.34992721e-01 -3.47056044e-01
   5.57027295e-02 -7.78633341e-01 -2.14820623e-01  9.93454056e-01
   7.11871289e-01  7.45091432e-01 -5.94164649e-01]
 [ 1.74678052e-02  2.17556125e-01 -1.55326435e-03  2.30

In [45]:
y_pred= model.predict(unlabel)
y_pred

array([5, 4, 3, 3])

In [46]:
lbl.inverse_transform(y_pred)

array(['SKY', 'PATH', 'GRASS', 'GRASS'], dtype=object)